# POO en Python

Temas incluidos:

1. Encapsulamiento y propiedades
2. Métodos especiales (Dunder methods)
3. Composición vs Herencia
4. Clases abstractas y métodos abstractos
5. Métodos estáticos y de clase
6. Otros conceptos y buenas prácticas

## 1. Encapsulamiento y propiedades

### ¿Qué es encapsulamiento?

Encapsulamiento es el principio de ocultar el estado interno de un objeto y exponer sólo lo necesario a través de una interfaz pública. En lenguajes como Java o C++ existen modificadores de acceso (`private`, `protected`, `public`) que aplican restricciones estrictas. Python adopta una filosofía diferente: **convenciones sobre restricciones**. No hay verdaderos atributos privados, pero sí formas de indicar que un atributo o método es de uso interno.

### Convenciones en Python

- `_atributo`: indica que es 'privado' por convención (no hay enforcement). Es un aviso para otros desarrolladores.
- `__atributo` (doble guión bajo inicial): activa name mangling: Python renombra internamente el atributo para evitar colisiones en subclases. No es seguridad real, pero dificulta el acceso accidental.
- No uses el doble guión bajo a menos que necesites evitar colisiones con subclases; el guión bajo simple suele ser suficiente.

### `@property` -> getters y setters

En Python se evita el uso explícito de `get_` y `set_` cuando es posible. El decorador `@property` permite exponer un método como si fuera un atributo. Esto permite validar asignaciones o calcular valores bajo demanda sin cambiar la API pública de la clase.

#### Ventajas:

- Mantener una interfaz limpia (acceso mediante `obj.atributo`).
- Poder añadir validaciones sin romper código que ya usa la clase.

A continuación viene un ejemplo práctico y extensos comentarios.


In [ ]:
"""
Ejemplo: CuentaBancaria con atributos 'privados' y @property
"""

class CuentaBancaria:
    """Clase que simula una cuenta bancaria simple.

    Atributos:
    - _titular: nombre del titular (convención: privado)
    - _saldo: saldo interno (no acceder directamente desde fuera)
    - currency: moneda pública (atributo de clase)
    """

    currency = 'EUR'  # atributo de clase

    def __init__(self, titular: str, saldo_inicial: float = 0.0):
        # usamos guión bajo para indicar que son internos
        self._titular = titular
        self._saldo = 0.0
        # Usamos la propiedad para validar el saldo inicial
        self.saldo = saldo_inicial

    # Propiedad 'saldo' — getter
    @property
    def saldo(self): 
        """Devuelve el saldo actual. Se podría añadir registro o lógica adicional aquí."""
        return self._saldo

    # Setter para saldo — valida no asignar saldos negativos
    @saldo.setter
    def saldo(self, value: float):
        if not isinstance(value, (int, float)):
            raise TypeError('El saldo debe ser numérico')
        if value < 0:
            raise ValueError('No se puede asignar saldo negativo')
        self._saldo = float(value)

    def ingresar(self, cantidad: float):
        if cantidad <= 0:
            raise ValueError('La cantidad a ingresar debe ser positiva')
        self._saldo += cantidad
        return self._saldo

    def retirar(self, cantidad: float):
        if cantidad <= 0:
            raise ValueError('La cantidad a retirar debe ser positiva')
        if cantidad > self._saldo:
            raise ValueError('Saldo insuficiente')
        self._saldo -= cantidad
        return self._saldo

    def __str__(self):
        return f"Cuenta({self._titular}: {self._saldo} {self.currency})"

# Demo ejecutable
cuenta = CuentaBancaria('Ana', 100.0)
print(cuenta)
print('Saldo actual (propiedad):', cuenta.saldo)
cuenta.ingresar(50)
print('Después de ingresar 50:', cuenta.saldo)
try:
    cuenta.saldo = -10
except Exception as e:
    print('Error al asignar saldo negativo:', e)

# mostrar que _saldo no está estrictamente protegido
print('Acceso directo a _saldo (no recomendado):', cuenta._saldo)


## 2. Métodos estáticos y de clase

### Definición y diferencias

- **Método de instancia**: recibe `self` y puede acceder al estado de la instancia.
- **Método de clase** (`@classmethod`): recibe `cls` (la clase) y es útil para fábricas o para modificar comportamiento que dependa de la clase.
- **Método estático** (`@staticmethod`): no recibe `self` ni `cls`. Es una función agrupada lógicamente dentro de la clase.

Cuándo usar cada uno:

- Usa `@staticmethod` para utilidades que pertenecen al concepto de la clase pero no necesitan acceder a la instancia ni a la clase.
- Usa `@classmethod` para métodos que creen instancias alternativas (constructores alternativos) o que actúen sobre la clase.

Ejemplo práctico con una clase `Empleado`.


In [ ]:
class Empleado:
    _next_id = 1  # atributo de clase para asignar ids crecientes

    def __init__(self, nombre: str, email: str):
        self.id = Empleado._next_id
        Empleado._next_id += 1
        self.nombre = nombre
        self.email = email

    @staticmethod
    def validar_email(email: str) -> bool:
        # implementación sencilla
        return '@' in email and '.' in email

    @classmethod
    def desde_string(cls, s: str):
        # formato: 'nombre|email'
        nombre, email = s.split('|')
        return cls(nombre.strip(), email.strip())

    def __str__(self):
        return f'Empleado(id={self.id}, nombre={self.nombre}, email={self.email})'

# Demo  
print(Empleado.validar_email('ana@example.com'))
emp = Empleado.desde_string('Luis | luis@mail.com')
print(emp)
print('Siguiente id global:', Empleado._next_id)


## 3. Métodos especiales (Dunder methods)

Los dunder methods (double-underscore methods) permiten que objetos personalizados interactúen con las construcciones nativas del lenguaje. Implementar estos métodos dota a nuestras clases de comportamiento natural: comparación, suma, iteración, conversión a cadena, etc.

Algunos muy útiles:

- `__str__`, `__repr__` — representación legible y no ambigua
- `__len__` — respuesta a `len(obj)`
- `__iter__`, `__next__` — para iteradores
- `__getitem__`, `__setitem__` — para indexación
- `__add__`, `__sub__` — operadores aritméticos
- `__eq__`, `__lt__`, etc. — comparaciones

Ejemplo: implementaremos una clase `Vector` con varios dunder methods y veremos cómo se comporta con operaciones nativas.


In [ ]:
from math import sqrt

class Vector:
    """Vector simple de dimensión n, con algunos métodos especiales implementados."""
    def __init__(self, coords):
        self.coords = list(coords)

    def __len__(self):
        return len(self.coords)

    def __repr__(self):
        return f"Vector({self.coords})"

    def __str__(self):
        return 'Vector de dimensión ' + str(len(self)) + ' -> ' + str(self.coords)

    def __eq__(self, other):
        if not isinstance(other, Vector):
            return NotImplemented
        return self.coords == other.coords

    def __add__(self, other):
        if not isinstance(other, Vector):
            return NotImplemented
        if len(self) != len(other):
            raise ValueError('Dimensiones diferentes')
        return Vector([a + b for a, b in zip(self.coords, other.coords)])

    def magnitude(self):
        return sqrt(sum(c*c for c in self.coords))

# Demo
v1 = Vector([1,2,3])
v2 = Vector([4,5,6])
print(v1)
print('len(v1)=', len(v1))
print('v1 + v2 =', v1 + v2)
print('v1 == v2?', v1 == v2)
print('magnitud v1=', v1.magnitude())


## 4. Composición vs Herencia

### Herencia

La herencia permite crear una nueva clase a partir de otra, heredando atributos y métodos. Es útil cuando existe una relación "es-un" (is-a).

Ventajas:
- Reutilización de código.
- Polimorfismo (mismo interfaz con implementaciones distintas).

Inconvenientes:
- Puede llevar a jerarquías complicadas.
- Fragilidad: cambios en la clase base afectan a las subclases.

### Composición

La composición construye objetos complejos a partir de otros objetos (relación "tiene-un", has-a). Es más flexible y suele ser preferible cuando no existe claramente una relación "es-un".

### Principio: Preferir composición sobre herencia

No es una regla absoluta, pero muchas veces es más seguro y flexible usar composición.

### Ejemplos comparativos


In [ ]:
# Herencia: Vehiculo -> Coche
class Vehiculo:
    def __init__(self, marca):
        self.marca = marca
    def mover(self):
        return 'El vehículo se mueve'

class Coche(Vehiculo):
    def __init__(self, marca, ruedas=4):
        super().__init__(marca)
        self.ruedas = ruedas
    def mover(self):
        return f'El coche de marca {self.marca} avanza sobre {self.ruedas} ruedas'

c = Coche('Toyota')
print(c.mover())
print(isinstance(c, Vehiculo))


### Composición: Coche tiene un Motor


In [1]:
class Motor:
    def __init__(self, potencia):
        self.potencia = potencia
    def arrancar(self):
        return 'Motor arrancado con potencia ' + str(self.potencia)

class CocheComp:
    def __init__(self, marca, motor: Motor):
        self.marca = marca
        self.motor = motor
    def mover(self):
        return f'Coche {self.marca}: ' + self.motor.arrancar()

m = Motor(120)
c2 = CocheComp('Seat', m)
print(c2.mover())
print('C2 tiene motor?', isinstance(c2.motor, Motor))


Coche Seat: Motor arrancado con potencia 120
C2 tiene motor? True


## 5. Clases abstractas y métodos abstractos

Las clases abstractas se usan para definir una interfaz común que las subclases deben implementar. En Python se usan con el módulo `abc`:

- `abc.ABC` como clase base
- `@abstractmethod` para marcar métodos que deben implementarse

Ventajas:
- Forzar una interfaz clara en sistemas modulares.
- Detectar errores de diseño en tiempo de definición (no se pueden instanciar clases abstractas).

Ejemplo práctico: sensores biomédicos, útil para bioinformática.


In [2]:
from abc import ABC, abstractmethod

class Secuenciador(ABC):
    """Interfaz base para secuenciadores de ADN/ARN"""

    @abstractmethod
    def leer_fragmento(self):
        """Leer un fragmento de secuencia"""
        pass

    @abstractmethod
    def calidad(self):
        """Calcular métrica de calidad de la lectura"""
        pass


class SecuenciadorNanopore(Secuenciador):
    def __init__(self, id_maquina):
        self.id = id_maquina
        self._ruido = 0.01  # error base

    def leer_fragmento(self):
        # ejemplo simplificado: devolver cadena aleatoria
        import random
        bases = ["A", "C", "G", "T"]
        return "".join(random.choice(bases) for _ in range(10))

    def calidad(self):
        # calidad ficticia inversa al ruido
        return 1.0 - self._ruido


# Intento de instanciar clase abstracta falla
try:
    s = Secuenciador()
except Exception as e:
    print("Error al instanciar clase abstracta Secuenciador:", e)

nano = SecuenciadorNanopore("NP-01")
print("Lectura ejemplo:", nano.leer_fragmento())
print("Calidad de la lectura:", nano.calidad())



Error al instanciar clase abstracta Secuenciador: Can't instantiate abstract class Secuenciador without an implementation for abstract methods 'calidad', 'leer_fragmento'
Lectura ejemplo: AACTTCGACT
Calidad de la lectura: 0.99


## 6. Otros conceptos y buenas prácticas

- **Polimorfismo**: varias clases implementan la misma interfaz y pueden usarse indistintamente.
- **Documentación**: usa docstrings en clases y métodos, y muestra ejemplos en doctest cuando tenga sentido.
- **Separación de responsabilidades**: cada clase debe tener una única responsabilidad (Single Responsibility Principle).
- **Testing**: escribe tests unitarios para las clases (pytest/unittest). Las clases con estado requieren tests que verifiquen cambios de estado.
